# 딥러닝 특강
## Deep Learning Practice 


#### Source from https://github.com/keai-kaist/CS492F-Spring

---

### 2-4. Save and restore models

Model progress can be saved during—and after—training. This means a model can resume where it left off and avoid long training times. Saving also means you can share your model and others can recreate your work. When publishing research models and techniques, most machine learning practitioners share:
- code to create the model, and
- the trained weights, or parameters, for the model

Sharing this data helps others understand how the model works and try it themselves with new data.

#### Setup

Install and import TensorFlow and dependencies:

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
    pass

!pip install -q pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

#### Get an example dataset

To demonstrate how to save and load weights, you'll use the MNIST dataset. To speed up these runs, use the first 1000 examples:

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

#### Define model 
Start by building a simple sequential model:

In [ ]:
# Define a simple sequential model
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, activation='softmax')
      ])
    
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    
    return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

#### Save checkpoints during training (Save weights only)

You can use a _trained model_ without having to retrain it, or pick-up training where you left off—in case the training process was interrupted. The [`tf.keras.callbacks.ModelCheckpoint`](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/callbacks/ModelCheckpoint) callback allows to continually save the model both during and at the end of training.

##### Checkpoint callback usage
Create a tf.keras.callbacks.ModelCheckpoint callback that **_saves weights only_** during training:

In [ ]:
import os

ckpt_path_for_t1 = "checkpoints/training_1/cp.ckpt"
ckpt_dir_for_t1 = os.path.dirname(ckpt_path_for_t1)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=ckpt_path_for_t1,
                                                 save_weights_only=True,
                                                 save_best_only=True,
                                                 verbose=2)

In [ ]:
# Train the model with the new callback
model.fit(train_images, 
         train_labels,
         epochs=10,
         validation_data=(test_images, test_labels),
         callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

This creates a single collection of TensorFlow checkpoint files that are updated at the end of each epoch:

In [ ]:
!ls {ckpt_dir_for_t1}

Create a new, untrained model. When restoring a model from weights-only, you **must have a model with the same architecture as the original model.** Since it's the same model architecture, **you can share weights** despite that it's a different instance of the model.

Now rebuild a fresh, untrained model, and evaluate it on the test set. An untrained model will perform at chance levels (~10% accuracy):

In [ ]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

Then load the weights from the checkpoint and re-evaluate:

In [ ]:
# Loads the weights
model.load_weights(ckpt_path_for_t1)

# Re-evaluate the model
loss,acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

##### Manually save weights

You saw how to load the weights into a model. Manually saving them is just as simple with the `Model.save_weights` method. By default, tf.keras—and `save_weights` in particular—uses the TensorFlow checkpoints format with a .ckpt extension

In [ ]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint/cur_weights')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint/cur_weights')

# Evaluate the model
loss,acc = model.evaluate(test_images, test_labels, verbose=0)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

#### Save the entire model 

The model and optimizer can be saved to a file that contains both their _state (weights and variables)_ and the _model configuration_. This allows you to export a model so it can be used without access to the original Python code. Since the optimizer-state is recovered, you can resume training from exactly where you left off.

Saving a fully-functional model is very useful—you can load them in TensorFlow.js ([HDF5 in TensorFlow.js](https://www.tensorflow.org/js/tutorials/conversion/import_keras), [Saved Model in TensorFlow.js](https://www.tensorflow.org/js/tutorials/conversion/import_saved_model)) and then train and run them in web browsers, or convert them to run on mobile devices using TensorFlow Lite ([HDF5 in Python](https://www.tensorflow.org/lite/convert/python_api#exporting_a_tfkeras_file_), [Saved Model in Python](https://www.tensorflow.org/lite/convert/python_api#exporting_a_savedmodel_))

##### Save model as an HDF5 file

Keras also provides a basic save format using the [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format) standard. For our purposes, the saved model can be treated as a single binary blob:

In [ ]:
# Create a new model instance
model = create_model()

# Train the model
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file
save_dir = 'saved_models'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

model.save(os.path.join(save_dir, 'my_model.h5'))

Now, recreate the model from that file:

In [ ]:
# TODO: Recreate the exact same model, including its weights and the optimizer
new_model = 

# Show the model architecture
new_model.summary()

Check its accuracy:

In [ ]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=0)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

This technique saves everything:

- The weight values
- The model's configuration(architecture)
- The optimizer configuration

Keras saves models by inspecting the architecture. Currently, it is not able to save TensorFlow optimizers (from tf.train). When using those you will need to re-compile the model after loading, and you will lose the state of the optimizer.